In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
from vllm import LLM

/root/prime-rl/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-03 14:25:09 [__init__.py:239] Automatically detected platform cuda.


2025-06-03 14:25:11,257	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from src.zeroband.inference.pipeline import PipelineConfig, patch_model_load, setup_comm, setup_hooks

config = PipelineConfig(
    rank=1,
    world_size=2,
    iroh_seed=1,
    iroh_peer_id="ee1aa49a4459dfe813a3cf6eb882041230c7b2558469de81f87c9bf23bf10a03",
    connection_num_retries=3,
)

node = setup_comm(config)
patch_model_load(config)

06-03 14:25:13 [INFER] [INFO] Created node (ff87a0b0a3c7c0ce827e9cada5ff79e75a44a0633bfcb5b50f99307ddb26b337)
[2025-06-03T14:25:13Z ERROR iroh::discovery::pkarr] pkarr_publish; me=ff87a0b0a3
[2025-06-03T14:25:13Z ERROR tracing::span] magicsock;
06-03 14:25:13 [INFER] [INFO] Setting up outgoing connection to ee1aa49a4459dfe813a3cf6eb882041230c7b2558469de81f87c9bf23bf10a03
[2025-06-03T14:25:13Z ERROR iroh::discovery] discovery; me=ff87a0b0a3 node=ee1aa49a44
06-03 14:25:14 [INFER] [INFO] Outgoing connection to ee1aa49a4459dfe813a3cf6eb882041230c7b2558469de81f87c9bf23bf10a03 successful!
06-03 14:25:14 [INFER] [INFO] Waiting for incoming connection...
06-03 14:25:15 [INFER] [INFO] All connections successful!
06-03 14:25:15 [INFER] [INFO] Patching model init for pp.rank=1 in pp.world_size=2


In [3]:
llm = LLM(
    model="Qwen/Qwen3-0.6B",
    tensor_parallel_size=2,
    max_seq_len_to_capture=16384,
    max_model_len=16384,
    quantization=None,
    enforce_eager=True,
    disable_async_output_proc=True,
    download_dir="/alloc",
    dtype="bfloat16")

setup_hooks(config, llm, node)

INFO 06-03 14:25:22 [config.py:717] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
WARNING 06-03 14:25:22 [arg_utils.py:1658] --disable-async-output-proc is not supported by the V1 Engine. Falling back to V0. We recommend to remove --disable-async-output-proc from your config in favor of the V1 Engine.
INFO 06-03 14:25:23 [config.py:1770] Defaulting to use mp for distributed inference
INFO 06-03 14:25:23 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='Qwen/Qwen3-0.6B', speculative_config=None, tokenizer='Qwen/Qwen3-0.6B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir='/alloc', load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_d

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00, 12.23it/s]


INFO 06-03 14:25:30 [loader.py:458] Loading weights took 0.09 seconds
(VllmWorkerProcess pid=367169) INFO 06-03 14:25:30 [model_runner.py:1140] Model loading took 0.3628 GiB and 0.252847 seconds


INFO 06-03 14:25:30 [weight_utils.py:315] No model.safetensors.index.json found in remote.
(VllmWorkerProcess pid=367169) INFO 06-03 14:25:30 [loader.py:458] Loading weights took 0.09 seconds
(VllmWorkerProcess pid=367169) INFO 06-03 14:25:31 [model_runner.py:1140] Model loading took 0.3628 GiB and 0.461013 seconds
(VllmWorkerProcess pid=367169) Input
(VllmWorkerProcess pid=367169) hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
(VllmWorkerProcess pid=367169) residual is None
Input
hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
residual is None
(VllmWorkerProcess pid=367169) Layer 14
(VllmWorkerProcess pid=367169) hidden_states[-1][-5:]: [3.0625, -0.328125, -2.203125, 1.7734375, 1.546875]
(VllmWorkerProcess pid=367169) residual[-1][-5:]: [2.21875, -0.1982421875, 1.3203125, 0.451171875, -2.3125]
(VllmWorkerProcess pid=367169) Layer 15
(VllmWorkerProcess pid=367169) hidden

In [4]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    max_tokens=1,
    temperature=0.7,
    top_p=0.9,
    top_k=40,
    seed=42,
)
request_outputs = llm.generate("Hello, world!", sampling_params)
print(request_outputs[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s, est. speed input: 14.15 toks/s, output: 3.54 toks/s]

(VllmWorkerProcess pid=367169) Input
(VllmWorkerProcess pid=367169) hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
(VllmWorkerProcess pid=367169) residual is None
(VllmWorkerProcess pid=367169) Layer 14
(VllmWorkerProcess pid=367169) hidden_states[-1][-5:]: [0.1875, -0.28515625, -0.4453125, 1.5, 1.2265625]
(VllmWorkerProcess pid=367169) residual[-1][-5:]: [2.09375, 0.55078125, 0.314453125, -0.2255859375, -1.90625]
(VllmWorkerProcess pid=367169) Layer 15
(VllmWorkerProcess pid=367169) hidden_states[-1][-5:]: [-0.0234375, -0.75390625, 1.3125, -1.9296875, -2.53125]
(VllmWorkerProcess pid=367169) residual[-1][-5:]: [5.6875, 0.9453125, -0.416015625, 1.6875, -1.8515625]
(VllmWorkerProcess pid=367169) Layer 16
(VllmWorkerProcess pid=367169) hidden_states[-1][-5:]: [1.6796875, -0.482421875, 0.74609375, -3.34375, 0.38671875]
(VllmWorkerProcess pid=367169) residual[-1][-5:]: [3.515625, -2.59375, 2.5, -0.423828125, -3.5]
(VllmWorkerProces

Input
hidden_states[-1][-5:]: [0.0546875, 0.00909423828125, 0.011962890625, -0.01055908203125, 0.0159912109375]
residual is None
Got hidden_states and residuals (torch.Size([4, 1024]), torch.Size([4, 1024])) (16544 bytes)
hidden_states[-1][-5:]: [-0.279296875, 0.67578125, 0.46875, 1.046875, -0.09765625]
residuals[-1][-5:]: [-0.275390625, 0.265625, -0.0380859375, -0.166015625, 0.84375]
Layer 14
hidden_states[-1][-5:]: [0.1875, -0.28515625, -0.4453125, 1.5, 1.2265625]
residual[-1][-5:]: [1.4765625, 1.484375, 0.734375, 0.66796875, -1.171875]
Layer 15
hidden_states[-1][-5:]: [-0.0234375, -0.75390625, 1.3125, -1.9296875, -2.53125]
residual[-1][-5:]: [5.0625, 1.8828125, 0.00390625, 2.59375, -1.1171875]
Layer 16
hidden_states[-1][-5:]: [1.6796875, -0.482421875, 0.74609375, -3.34375, 0.38671875]
residual[-1][-5:]: [2.890625, -1.65625, 2.921875, 0.482421875, -2.78125]
Layer 17
hidden_states[-1][-5:]: [-1.15625, 1.9375, 1.046875, -2.1875, 0.88671875]
residual[-1][-5:]: [3.625, -2.25, 3.890625, -